<a href="https://colab.research.google.com/github/vanessailana/Medical-ChatBot-/blob/main/GPT_2_and_ClinicalBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import re
import numpy as np
#loading medical data from the disk
all_data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/questiondoc.csv')
     
all_data

,Unnamed: 0,answer,answer_author,question,question_text,tags,url
0,0,you can stay assured you are fine free of hiv....,Dr. Ayman Darrag,is my anti hiv test conclusive or need retest?,Is my Anti Hiv Test Conclusive or need retest?,['hiv test'],https://questiondoctors.com/is-my-anti-hiv-tes...
1,1,hi you are 100 % hiv free good luck,Dr Ahmed Fawzy,is my anti hiv test conclusive or need retest?,Is my Anti Hiv Test Conclusive or need retest?,['hiv test'],https://questiondoctors.com/is-my-anti-hiv-tes...
2,2,your time slots of the tests and the final res...,Dr.Honey,is my anti hiv test conclusive or need retest?,Is my Anti Hiv Test Conclusive or need retest?,['hiv test'],https://questiondoctors.com/is-my-anti-hiv-tes...
3,3,hi i see no labrum tear however i see acetabul...,Dr Ahmed Fawzy,i have some hip pain 9 weeks. had mra image re...,I have some hip pain 9 weeks. Had MRA image re...,['magnetic resonance angiography (mra)'],https://questiondoctors.com/i-have-some-hip-pa...
4,4,no visible tear in labrum but obviously calcif...,Dr. Ayman Darrag,i have some hip pain 9 weeks. had mra image re...,I have some hip pain 9 weeks. Had MRA image re...,['magnetic resonance angiography (mra)'],https://questiondoctors.com/i-have-some-hip-pa...
...,...,...,...,...,...,...,...
5674,5674,get it confirmed by doing venous doppler of bo...,Dr Mahaveer,pain in legs – varicose veins?,Question: Pain in Legs – Varicose veins?,[],https://questiondoctors.com/question-pain-in-l...
5675,5675,hi dear. there are very less chances of pregna...,Dr Mahaveer,headaches really tired all the time feeling na...,"Question: Headaches, really tired all the time...","[""don't know if its too early to take a pregna...",https://questiondoctors.com/question-headaches...
5676,5676,i’m a radiologist. may be an mri misread. when...,Dr. Erik Ramonov,46 year old male had stroke on right side of b...,Question: 46 year old male had stroke on right...,['stroke on right side of brain'],https://questiondoctors.com/question-46-year-o...
5677,5677,could be reaction to oxycodone…. ”the most fre...,Jason Roberge,46 year old male had stroke on right side of b...,Question: 46 year old male had stroke on right...,['stroke on right side of brain'],https://questiondoctors.com/question-46-year-o...


In [5]:

#function to preprocess tags post loading from disk
'''


The given code defines a Python function named preprocess_tags. This function takes a single argument tags,
 which is expected to be a string containing a comma-separated list of tags enclosed in square brackets.

The function first removes the square brackets from the tags string using string slicing (tags[1:-1]). 
It then removes any single quotes in the resulting string using the replace() method.

Finally, the function splits the string into a list of individual tags using the split() method with the argument ', ', 
which is a comma followed by a space. This list of tags is then converted to a NumPy array using the np.array() function 
and returned from the function.

Overall, this function preprocesses a string of tags by removing the square brackets and single quotes 
and splitting the tags into an array of individual tags.

By using 1:-1, we are telling Python to include all characters in the string except for the first and last ones.
'''
def preprocess_tags(tags):
  tags=tags[1:-1]
  tags=tags.replace("\'",'')
  return np.array(tags.split(', '))
     

In [6]:

#preprocessing the tags post loading from disk
all_data['tags']=all_data.tags.apply(lambda x: preprocess_tags(x))

In [7]:
import nltk
from nltk.tokenize import RegexpTokenizer
import re


def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    '''
    This line converts all characters in the string to lowercase, which is a common preprocessing
     step for natural language processing tasks. '''



  #This line removes any occurrences of the string "{html}" from the sentence using the replace() method.

    sentence=sentence.replace('{html}',"") 

    '''
    These lines use the re module to remove any HTML tags from the sentence. The regular expression '<.*?>' matches any string
     between < and > characters, which is typically used to represent HTML tags. The sub() method 
    replaces all instances of this pattern with an empty string, effectively removing the HTML tag
    '''

    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    '''
    This line removes any URLs (i.e., strings starting with "http" or "https") 
    from the sentence using another regular expression. '''

    rem_url=re.sub(r'http\S+', '',cleantext)

    #This line removes any numbers from the sentence using yet another regular expression.
    rem_num = re.sub('[0-9]+', '', rem_url)

    ''''
    hese lines tokenize the sentence into individual words using the RegexpTokenizer class from the nltk.tokenize module. The regular expression
     \w+ matches any sequence of
     one or more word characters (letters, digits, or underscores). 
    The tokenize() method then applies this regular expression to the rem_num string, returning a list of individual tokens.
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    return " ".join(tokens) '''
     
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    return " ".join(tokens)


all_data['questions_clean']=all_data['question_text'].apply(preprocess)
all_data['answers_clean']=all_data['answer'].apply(preprocess)


In [2]:
all_data

,Unnamed: 0,answer,answer_author,question,question_text,tags,url
0,0,you can stay assured you are fine free of hiv....,Dr. Ayman Darrag,is my anti hiv test conclusive or need retest?,Is my Anti Hiv Test Conclusive or need retest?,['hiv test'],https://questiondoctors.com/is-my-anti-hiv-tes...
1,1,hi you are 100 % hiv free good luck,Dr Ahmed Fawzy,is my anti hiv test conclusive or need retest?,Is my Anti Hiv Test Conclusive or need retest?,['hiv test'],https://questiondoctors.com/is-my-anti-hiv-tes...
2,2,your time slots of the tests and the final res...,Dr.Honey,is my anti hiv test conclusive or need retest?,Is my Anti Hiv Test Conclusive or need retest?,['hiv test'],https://questiondoctors.com/is-my-anti-hiv-tes...
3,3,hi i see no labrum tear however i see acetabul...,Dr Ahmed Fawzy,i have some hip pain 9 weeks. had mra image re...,I have some hip pain 9 weeks. Had MRA image re...,['magnetic resonance angiography (mra)'],https://questiondoctors.com/i-have-some-hip-pa...
4,4,no visible tear in labrum but obviously calcif...,Dr. Ayman Darrag,i have some hip pain 9 weeks. had mra image re...,I have some hip pain 9 weeks. Had MRA image re...,['magnetic resonance angiography (mra)'],https://questiondoctors.com/i-have-some-hip-pa...
...,...,...,...,...,...,...,...
5674,5674,get it confirmed by doing venous doppler of bo...,Dr Mahaveer,pain in legs – varicose veins?,Question: Pain in Legs – Varicose veins?,[],https://questiondoctors.com/question-pain-in-l...
5675,5675,hi dear. there are very less chances of pregna...,Dr Mahaveer,headaches really tired all the time feeling na...,"Question: Headaches, really tired all the time...","[""don't know if its too early to take a pregna...",https://questiondoctors.com/question-headaches...
5676,5676,i’m a radiologist. may be an mri misread. when...,Dr. Erik Ramonov,46 year old male had stroke on right side of b...,Question: 46 year old male had stroke on right...,['stroke on right side of brain'],https://questiondoctors.com/question-46-year-o...
5677,5677,could be reaction to oxycodone…. ”the most fre...,Jason Roberge,46 year old male had stroke on right side of b...,Question: 46 year old male had stroke on right...,['stroke on right side of brain'],https://questiondoctors.com/question-46-year-o...


In [8]:

#finding the lens of preprocessed questions and answers
all_data['question_len']=all_data['questions_clean'].apply(lambda x: len(x.split(' ')))
all_data['answer_len']=all_data['answers_clean'].apply(lambda x: len(x.split(' ')))
     

'''This code calculates and prints the 10th, 20th, 30th, ..., 
100th percentiles of the lengths of questions and answers in a dataset.
'''
#printing different percentiles of question and answer lengths
import numpy as np
for i in range(0,101,10):
  print(i,np.percentile(all_data.question_len,i),np.percentile(all_data.answer_len,i))
     

0 4.0 1.0
10 8.0 19.0
20 10.0 25.0
30 11.0 30.0
40 12.0 34.0
50 14.0 38.0
60 14.0 43.0
70 16.0 49.0
80 17.0 56.0
90 19.0 68.0
100 33.0 106.0


In [9]:

#since more than 100% percent of questions and about 99% of all answers lie within 500 words, we use 500 as the max length
all_data['short_question']=all_data.apply(lambda x: ' '.join(x.questions_clean.split(' ')[:33]) if x.question_len>33 else x.questions_clean ,axis=1)
all_data['short_answer']=all_data.apply(lambda x: ' '.join(x.answers_clean.split(' ')[:106]) if x.answer_len>106 else x.answers_clean ,axis=1)


In [10]:

#preparing the positive label dataset by taking the original question and original answer
all_dataset=all_data[['short_question','short_answer','tags']].copy()
all_dataset['label']=1.0
     

In [11]:
all_dataset.head(3)

,short_question,short_answer,tags,label
0,is my anti hiv test conclusive or need retest,you can stay assured you are fine free of hiv ...,[hiv test],1.0
1,is my anti hiv test conclusive or need retest,hi you are hiv free good luck,[hiv test],1.0
2,is my anti hiv test conclusive or need retest,your time slots of the tests and the final res...,[hiv test],1.0


In [12]:

#function to extract the negative samples using tags
def extract_negative_samples(question,tags):
  stop=False
  while (not stop):
    sample_row= all_dataset.sample()
    sample_tags=sample_row.tags.values[0]
    print(sample_tags)
    inter_tags=set(tags[0]).intersection(set(sample_tags))
    if len(inter_tags)==0:
      stop=True
  return sample_row


'''
This code defines a function called extract_negative_samples that takes two parameters: 
question and tags. The purpose of this function is to extract a negative sample, 
meaning a sample that does not contain any of the tags specified in the tags parameter.

The function uses a while loop to keep searching for a negative sample until it finds one. 
Inside the while loop, it randomly selects a row from a dataset called
 all_dataset using the sample() method. 
 
 It then extracts the tags associated with the selected row
  and stores them in a variable called sample_tags.

Next, the function uses the intersection() method to find the 
common tags between the tags parameter and the sample_tags. 
If there are no common tags, it sets the stop variable to True,
 indicating that a negative sample has been found.

Finally, the function returns the selected row as the 
negative sample using the return statement.

It's worth noting that the code assumes that all_dataset
 is a pandas DataFrame with a column named tags. 
 It also assumes that tags is a list of tags associated with the question.

'''

"\nThis code defines a function called extract_negative_samples that takes two parameters: \nquestion and tags. The purpose of this function is to extract a negative sample, \nmeaning a sample that does not contain any of the tags specified in the tags parameter.\n\nThe function uses a while loop to keep searching for a negative sample until it finds one. \nInside the while loop, it randomly selects a row from a dataset called\n all_dataset using the sample() method. \n \n It then extracts the tags associated with the selected row\n  and stores them in a variable called sample_tags.\n\nNext, the function uses the intersection() method to find the \ncommon tags between the tags parameter and the sample_tags. \nIf there are no common tags, it sets the stop variable to True,\n indicating that a negative sample has been found.\n\nFinally, the function returns the selected row as the \nnegative sample using the return statement.\n\nIt's worth noting that the code assumes that all_dataset\n 

In [13]:

#preparing the negative labelled dataset
from tqdm.notebook import tqdm
tqdm.pandas()
negative_labels=all_dataset.progress_apply(lambda x: pd.Series([x.short_question,extract_negative_samples(x.short_question,x.tags).short_answer.values[0],x.tags]),axis=1)
negative_labels['label']=-1.0
negative_labels.columns=['short_question','short_answer','tags','label']

  0%|          | 0/5679 [00:00<?, ?it/s]

Streaming output truncated to the last 5000 lines.
['i suffered a major left sided hemiparesis in july 04']
['is it tetanus']
['cervical intraepithelial neoplasia' 'cold coagulation for cin']
['dizziness and blurred vision']
['i was put on morphine and percocet and spinal injections for pain']
['ilia conduit operation']
['advice regarding thalassemia and pregnancy']
['bumps on her arm']
['i fractured my spine now have vision problems']
['chronic obstructive pulmonary disease (copd) and erectile dysfunction (ed)']
['sudden twitch/spasm feeling in my stomach']
['grand mal seizure']
['not sure my doctor is doing it right'
 'pernicious anaemia and require regular b12 injections']
['apple cider vinegar to relieve cold and flu']
['i have stomach issues fever diarrhea']
['withdrawl symtoms']
['paralysis in his left leg']
['stomach pain']
['i noticed a burning sensation on my side']
['']
['epidural for pain today in my lumbar']
['']
['died while at hospital']
['pregnant immediately after a mis

In [14]:

#displaying the negative labelled dataset
negative_labels

,short_question,short_answer,tags,label
0,is my anti hiv test conclusive or need retest,hi costochondritis is uncommon at your age thi...,[hiv test],-1.0
1,is my anti hiv test conclusive or need retest,hi it looks like infection in the parotid glan...,[hiv test],-1.0
2,is my anti hiv test conclusive or need retest,that would be a colic pain you can get an abdo...,[hiv test],-1.0
3,i have some hip pain weeks had mra image recen...,hi my opinion you have l l nerve intrapment i ...,[magnetic resonance angiography (mra)],-1.0
4,i have some hip pain weeks had mra image recen...,in my opinion this could be atopic dermatitis ...,[magnetic resonance angiography (mra)],-1.0
...,...,...,...,...
5674,question pain in legs varicose veins,hi you need to get a hrct chest chest xray bro...,[],-1.0
5675,question headaches really tired all the time f...,i n my opinion this xray shows mild osteoarthr...,"[""dont know if its too early to take a pregnan...",-1.0
5676,question year old male had stroke on right sid...,definitely it could be due to low fiber intake...,[stroke on right side of brain],-1.0
5677,question year old male had stroke on right sid...,right axis deviation in general means that the...,[stroke on right side of brain],-1.0


In [15]:

#concatenating the positive and negative labelled dataset to get the final labelled dataset.
all_data_with_labels=pd.concat([all_dataset,negative_labels],axis=0)
all_data_with_labels.shape
     

(11358, 4)

In [16]:
#splitting the data into train and validation
from sklearn.model_selection import train_test_split
train, validation = train_test_split(all_data_with_labels, test_size=0.2,random_state=42,shuffle=True,stratify=all_data_with_labels.label)
     

#saving train and validation data to disk
train.to_csv('train_data_chatbot.csv',index=False)
validation.to_csv('validation_data_chatbot.csv',index=False)
     

In [17]:
train=pd.read_csv('train_data_chatbot.csv')
validation=pd.read_csv('validation_data_chatbot.csv')


In [18]:

#displaying the train label counts
train.label.value_counts()

 1.0    4543
-1.0    4543
Name: label, dtype: int64

In [19]:

#extracting the short answers and questions from the train data
questions=train['short_question']
answers=train['short_answer']
labels=train['label']
     

In [21]:

#installing the transformers library
!pip install transformers
     

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 114.1 MB/s eta 0:00:00


In [29]:
import torch
from transformers import AutoTokenizer, AutoModel

# Load the ClinicalBERT tokenizer and model
biobert_tokenizer = AutoTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')
biobert_model = AutoModel.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [30]:
import tensorflow as tf

MAX_LENGTH = 512
# Tokenize, filter and pad sentences
def tokenize_and_filter(questions, answers):
  tokenized_questions, tokenized_answers = [], []
  
  for (question, answer) in zip(questions, answers):
    # generating sequences
    tokenized_question =  biobert_tokenizer.encode(question)
    tokenized_answer = biobert_tokenizer.encode(answer)
    tokenized_questions.append(tokenized_question)
    tokenized_answers.append(tokenized_answer)

  # padding the sequences
  tokenized_questions = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_questions, maxlen=MAX_LENGTH, padding='post')
  tokenized_answers = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_answers, maxlen=MAX_LENGTH, padding='post')
  
  return tokenized_questions, tokenized_answers

#tokenizing and padding the train questions and answers
questions, answers = tokenize_and_filter(questions, answers)
    

Why do we pad, filter, and tokenize sentences?

We pad, filter, and tokenize sentences for several reasons:

Standardize input length: Neural networks require fixed input size to work efficiently. By padding shorter sentences with a padding token to match the length of longer sentences, we can create input sequences with consistent lengths.

Reduce data noise: Tokenization involves breaking down text into individual words or subwords. This process removes unnecessary data such as punctuation marks, extra spaces, and other special characters, which could cause noise and negatively affect the model's performance.

Encode input data: Machine learning models work on numerical data. Tokenization allows us to convert text data into numerical representations such as vectors, which can be easily processed by the model.

Preserve semantic meaning: Tokenization not only encodes words numerically but also takes into account their context. This helps preserve the semantic meaning of the original text and enables the model to understand the relationships between words in a sentence.

Improve model performance: By standardizing the input data, removing noise, and encoding it numerically, we can improve the performance of the machine learning models by reducing the impact of irrelevant features and enabling more efficient computation.

Overall, tokenization, filtering, and padding are essential steps in natural language processing and are crucial for building accurate and efficient machine learning models.

In [32]:
'''
This code creates two mask matrices train_question_mask and train_answer_mask for the training set of a natural language processing task. These mask matrices are used to indicate which tokens in the input text sequences are actual words and which ones are padding tokens.

Here is how the code works:

train_question_mask is created by iterating over each question in the questions list and creating a new list of 1s and 0s for each question. For each token in a question, a 1 is added to the new list if the token is not equal to 0 (which is assumed to be a padding token), otherwise a 0 is added. This process results in a mask matrix for each question where 1s represent actual words and 0s represent padding tokens.
train_answer_mask is created in the same way as train_question_mask, except it is created for each answer in the answers list.
Overall, the purpose of these mask matrices is to tell the neural network model which tokens in the input text sequences are actual words and which ones are padding tokens. This is important because when the neural network is processing the input sequences, it needs to know which tokens to pay attention to and which ones to ignore. By masking out the padding tokens, the neural network can focus only on the actual words
 in the input sequences and produce more accurate predictions.

 '''
#preparing the question mask and the answer mask of the train dataset
train_question_mask=[[1 if token!=0 else 0 for token in question] for question in questions]
train_answer_mask=[[1 if token!=0 else 0 for token in answer] for answer in answers]
     


In [33]:

#extracting the short answers and questions from the validation data
val_questions=validation['short_question']
val_answers=validation['short_answer']
val_labels=validation['label']
     

In [34]:

#tokenizing and padding the validation question and answers
val_questions, val_answers = tokenize_and_filter(val_questions, val_answers)

In [35]:

#preparing the question and answer mask for the validation data
val_question_mask=[[1 if token!=0 else 0 for token in question] for question in val_questions]
val_answer_mask=[[1 if token!=0 else 0 for token in answer] for answer in val_answers]
     

In [36]:

import tensorflow as tf
BATCH_SIZE = 2
BUFFER_SIZE = 20000
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'question': questions,
        'answer': answers,
        'question_mask':train_question_mask,
        'answer_mask':train_answer_mask
    },
    {
        'label': labels.values
    },
))

# dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE,drop_remainder=True)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
     

In [37]:
import tensorflow as tf
BATCH_SIZE = 2
BUFFER_SIZE = 20000
val_dataset = tf.data.Dataset.from_tensor_slices((
    {
        'question': val_questions,
        'answer': val_answers,
        'question_mask':val_question_mask,
        'answer_mask':val_answer_mask
    },
    {
        'label': val_labels.values
    },
))

val_dataset = val_dataset.cache()
val_dataset = val_dataset.shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE,drop_remainder=True)
val_dataset = val_dataset.prefetch(tf.data.experimental.AUTOTUNE)
     